# Esercizio 1 - Definizioni

In [70]:
import pandas as pd
import nltk

### Leggi il file di input

Salvo le varie definizioni degli studenti in un dizionario con:
- chiave: parola (tra Emotion, Person, Revenge, Brick)
- valore: lista di definizioni date dagli studenti

In [71]:
df = pd.read_excel('../data/definizioni.xlsx')
students = df.columns[1:].tolist()
defs = df[df.columns[0]].tolist()
def_dict = {}
for definition in defs:
    def_dict[definition] = []
for student in students:
    for definition in defs:
        def_dict[definition].append(df[student][df[df.columns[0]] == definition].values[0])

def_dict

{'Emotion': ['Range of concepts human beings feel in certain situations',
  'Something you can feel',
  'Something that an animal can feel',
  'something you think that makes you feel good or bad',
  'Human sensation arising from the form of feelings',
  'State of mind that a living being can percieve',
  'Feeling that a human or an animal can express towards others',
  'A sentiment that a living entity can feel and express throw words and their body.',
  'what you feel in a certain moment',
  "feeling experienced by sentient beings and cause by events happening in the outside world as well as in the being's ming",
  nan,
  nan,
  'sentimental reation to an action',
  'something you can feel',
  'what a human being is feeling, eg happyness, sadness, love',
  'general concept of feeling',
  'A feeling deriving from human life',
  'an human sensation ',
  'An animal feeling',
  'Emotion is a psychic state',
  'something that a human being can feel',
  'Human sensation that can affect som

### Ripulisci il file da stopwords e punteggiatura

In [54]:
# Remove punctuation and stopwords
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.WordNetLemmatizer()
tokens = []
for definition in defs:
    if df.iloc[definition] is not None:
        print(definition)

        # tokens = [word.lower() for word in tokenizer.tokenize(df[col].tolist()) if word.lower() not in stopwords]
        # df[col] = set([lemmatizer.lemmatize(word.lower()) for word in tokenizer.tokenize(df[col].tolist()) if word.lower() not in stopwords])


TypeError: Cannot index by location index with a non-integer key